In [ ]:
import tensorflow as tf

import numpy as np
import os
import time

In [ ]:
import pandas as pd
! pip install -q kaggle
from google.colab import files

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d kingburrito666/better-donald-trump-tweets

!unzip \*.zip  && rm *.zip


data = pd.read_csv('Donald-Tweets!.csv')['Tweet_Text']

new_list = []
for tweet in data:
  if 'http' in tweet or ':' in tweet:
    pass
  else:
    new_list.append(tweet)

text = ''.join(new_list)



# text = ""

# import glob, os
# for file in glob.glob("*.txt"):
#   text = text + open(file, "r").read()
#   print(file)

  0% 0.00/549k [00:00<?, ?B/s]
100% 549k/549k [00:00<00:00, 90.8MB/s]
Archive:  better-donald-trump-tweets.zip
replace Donald-Tweets!.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
# The unique characters in the file
vocab = sorted(set(text))

87 unique characters


In [ ]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [ ]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)


B
u
s
y
 


In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

'Busy day planned in New York. Will soon be making some very important decisions on the people who wil'
'l be running our government!Love the fact that the small groups of protesters last night have passion'
' for our great country. We will all come together and be proud!Just had a very open and successful pr'
'esidential election. Now professional protesters, incited by the media, are protesting. Very unfair!A'
' fantastic day in D.C. Met with President Obama for first time. Really good meeting, great chemistry.'


In [ ]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(100),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [ ]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)

(64, 100, 87) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (64, None, 256)           22272     
_________________________________________________________________
gru_6 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
gru_7 (GRU)                  (64, None, 1024)          6297600   
_________________________________________________________________
dense_8 (Dense)              (64, None, 100)           102500    
_________________________________________________________________
dense_9 (Dense)              (64, None, 87)            8787      
Total params: 10,369,463
Trainable params: 10,369,463
Non-trainable params: 0
_________________________________________________________________


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 87)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.466543


In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS=100

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
51/51 [==============================] - 7s 143ms/step - loss: 3.5315
Epoch 2/100
51/51 [==============================] - 7s 146ms/step - loss: 2.7589
Epoch 3/100
51/51 [==============================] - 8s 150ms/step - loss: 2.4334
Epoch 4/100
51/51 [==============================] - 8s 152ms/step - loss: 2.1806
Epoch 5/100
51/51 [==============================] - 8s 150ms/step - loss: 1.9370
Epoch 6/100
51/51 [==============================] - 7s 146ms/step - loss: 1.7315
Epoch 7/100
51/51 [==============================] - 7s 145ms/step - loss: 1.5811
Epoch 8/100
51/51 [==============================] - 7s 144ms/step - loss: 1.4658
Epoch 9/100
51/51 [==============================] - 7s 143ms/step - loss: 1.3727
Epoch 10/100
51/51 [==============================] - 7s 143ms/step - loss: 1.2961
Epoch 11/100
51/51 [==============================] - 7s 144ms/step - loss: 1.2240
Epoch 12/100
51/51 [==============================] - 7s 144ms/step - loss: 1.1541
Epoch 13/100


In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_100'

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (1, None, 256)            22272     
_________________________________________________________________
gru_8 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
gru_9 (GRU)                  (1, None, 1024)           6297600   
_________________________________________________________________
dense_10 (Dense)             (1, None, 100)            102500    
_________________________________________________________________
dense_11 (Dense)             (1, None, 87)             8787      
Total params: 10,369,463
Trainable params: 10,369,463
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"Sleepy Joe Biden "))

Sleepy Joe Biden day plaining to New Hampshire. Great honor!Lightweight choker Marco Rubio looks like a little boy on stage. Not presidential material!.@oreillyfactor  The FBI not big head by TIME AMERICA GREAT AGAIN! MAKE AMERICA SAFE AGAIN!People very untalentedrible represents the only one. I will keep doing, but not worth it!I will raised Union Leader refuses to comment as to why they were kicked out of the ABC News debate like a dog. Why dont know his family used private eminent! Will be leave for him informidani- in 2006 ! It is the that I had 17 opponents and she just had the worst jobs report since 2010.The Mayor of San Jose the failing @nytimes write the real anatime has made that the WALL was very nece!Sart Like Trump Convention victory. She is a winner!Watching other networks and local news. Really good night!.@KatrinaCarson the airhe۪s a clown with zero credibilitya Bush &amp; massive setback - no longer watch, said "Trump should info many people who know nothing about it a